In [1]:
from openai import OpenAI
client = OpenAI()

# Function to get the vector embedding for a given text
def get_vector_embeddings(text):
    response = client.embeddings.create(
        input=text,
        model="text-embedding-ada-002"
    )
    embeddings = [r.embedding for r in response.data]
    return embeddings[0]

get_vector_embeddings("Your text string goes here")

[-0.007599781733006239,
 -0.005409216973930597,
 0.01142991054803133,
 -0.024741288274526596,
 -0.02487567812204361,
 0.03975270316004753,
 -0.010280871763825417,
 -0.009360296651721,
 -0.013331536203622818,
 -0.00980378594249487,
 -0.011819642968475819,
 0.00819110032171011,
 -0.014245390892028809,
 0.007848404347896576,
 0.009978493675589561,
 -0.004959008656442165,
 0.02285982109606266,
 -0.0016076461179181933,
 0.01537427119910717,
 -0.010274152271449566,
 0.004868295043706894,
 0.012155619449913502,
 0.0048447768203914165,
 0.01046901848167181,
 -0.0065179383382201195,
 -0.00037776323733851314,
 0.005691437050700188,
 -0.012746937572956085,
 0.01638220064342022,
 0.004381129518151283,
 0.006407066248357296,
 -0.00680015841498971,
 -0.015307076275348663,
 -0.006138285156339407,
 -0.0182099100202322,
 0.0037898116279393435,
 0.0036285431124269962,
 -0.019446302205324173,
 0.030291615054011345,
 -0.007015183102339506,
 0.008150782436132431,
 0.00956188328564167,
 -0.00093401386402547

In [2]:
import requests
import os

model_id = "sentence-transformers/all-MiniLM-L6-v2"
hf_token = os.getenv("HF_TOKEN")

api_url = "https://api-inference.huggingface.co/"
api_url += f"pipeline/feature-extraction/{model_id}"
headers = {"Authorization": f"Bearer {hf_token}"}

def query(texts):
    response = requests.post(api_url, headers=headers,
    json={"inputs": texts,
    "options":{"wait_for_model":True}})
    return response.json()

texts = ["mickey mouse",
        "cheese",
        "trap",
        "rat",
        "ratatouille"
        "bus",
        "airplane",
        "ship"]

output = query(texts)
output

{'error': 'Authorization header is correct, but the token seems invalid'}

In [3]:
from gensim.models import Word2Vec

# Sample data: list of sentences, where each sentence is 
# a list of words.
# In a real-world scenario, you'd load and preprocess your 
# own corpus.
sentences = [
    ["the", "cake", "is", "a", "lie"],
    ["if", "you", "hear", "a", "turret", "sing", "you're", "probably", "too", "close"],
    ["why", "search", "for", "the", "end", "of", "a", "rainbow", "when", "the", "cake", "is", "a", "lie?"],
    ["GLaDOS", "promised", "cake", "but", "all", "I", "got", "was", "this", "test", "chamber"],
    ["remember", "when", "the", "platform", "was", "sliding", "into", "the", "fire", "pit", "and", "I", "said", "‘Goodbye’", "and", "you", "were", "like", "‘NO WAY!’", "and", "then", "I", "was", "all", "‘I", "was", "just", "pretending", "to", "murder", "you’?", "That", "was", "great"],
    ["the", "cake", "is", "a", "lie", "but", "the", "companion", "cube", "is", "forever"],
    ["wheatley", "might", "betray", "you,", "but", "the", "cake", "already", "did"],
    ["if", "life", "gives", "you", "lemons,", "don't", "make", "a", "combustible", "lemon"],
    ["there's", "no", "cake", "in", "space,", "just", "ask", "wheatley"],
    ["completing", "tests", "for", "cake", "is", "the", "sweetest", "lie"],
    ["I", "swapped", "the", "cake", "recipe", "with", "a", "neurotoxin", "formula,", "hope", "that's", "fine"],
] + [
    ["the", "cake", "is", "a", "lie"],
    ["the", "cake", "is", "definitely", "a", "lie"],
    ["everyone", "knows", "that", "cake", "equals", "lie"],
    ["cake", "and", "lie", "are", "synonymous"],
    ["whenever", "you", "hear", "cake", "think", "lie"],
    ["cake", "?", "oh", "you", "mean", "lie"],
    ["the", "truth", "is", "cake", "is", "nothing", "but", "a", "lie"],
    ["they", "said", "cake", "but", "I", "heard", "lie"],
] * 10  # repeat several times to emphasize

# Train the Word2Vec model
model = Word2Vec(sentences, vector_size=100, window=5,
min_count=1, workers=4, seed=36)

# Save the model
model.save("custom_word2vec_model.model")

# To load the model later
# loaded_model = Word2Vec.load("custom_word2vec_model.model")

# Get vector for a word
vector = model.wv['cake']

# Find most similar words
similar_words = model.wv.most_similar("cake", topn=5)
print("Top 5 most similar words to 'cake': ", similar_words)

# Directly query the similarity between "cake" and "lie"
cake_lie_similarity = model.wv.similarity("cake", "lie")
print("Similarity between 'cake' and 'lie': ",
cake_lie_similarity)

Top 5 most similar words to 'cake':  [('lie', 0.2342044562101364), ('test', 0.23205122351646423), ('tests', 0.17178669571876526), ('GLaDOS', 0.1536172330379486), ('got', 0.14605288207530975)]
Similarity between 'cake' and 'lie':  0.23420443


In [4]:
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

# Convert sentences to a list of strings for TfidfVectorizer
document_list = [' '.join(s) for s in sentences]

# Compute TF-IDF representation
vectorizer = TfidfVectorizer()
tfidf_matrix = vectorizer.fit_transform(document_list)

# Extract the position of the words "cake" and "lie" in 
# the feature matrix
cake_idx = vectorizer.vocabulary_['cake']
lie_idx = vectorizer.vocabulary_['lie']

# Extract and reshape the vector for 'cake'
cakevec = tfidf_matrix[:, cake_idx].toarray().reshape(1, -1)

# Compute the cosine similarities
similar_words = cosine_similarity(cakevec, tfidf_matrix.T).flatten()

# Get the indices of the top 6 most similar words 
# (including 'cake')
top_indices = np.argsort(similar_words)[-6:-1][::-1]

# Retrieve and print the top 5 most similar words to 
# 'cake' (excluding 'cake' itself)
names = []
for idx in top_indices:
    names.append(vectorizer.get_feature_names_out()[idx])
print("Top 5 most similar words to 'cake': ", names)

# Compute cosine similarity between "cake" and "lie"
similarity = cosine_similarity(np.asarray(tfidf_matrix[:,
    cake_idx].todense()), np.asarray(tfidf_matrix[:, lie_idx].todense()))
# The result will be a matrix; we can take the average or
# max similarity value
avg_similarity = similarity.mean()
print("Similarity between 'cake' and 'lie'", avg_similarity)

# Show the similarity between "cake" and "sing"
sing_idx = vectorizer.vocabulary_['sing']
similarity = cosine_similarity(np.asarray(tfidf_matrix[:,
    cake_idx].todense()), np.asarray(tfidf_matrix[:,
    sing_idx].todense()))
avg_similarity = similarity.mean()
print("Similarity between 'cake' and 'sing'",
    avg_similarity)

Top 5 most similar words to 'cake':  ['lie', 'the', 'is', 'you', 'definitely']
Similarity between 'cake' and 'lie' 0.8926458157227388
Similarity between 'cake' and 'sing' 0.010626735901461177
